I will attempt my filter for the first time. 

In [604]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchsummary import summary


In [605]:
class Upsample(nn.Module):
    def __init__(self,  scale_factor):
        super(Upsample, self).__init__()
        self.scale_factor = scale_factor
    def forward(self, x):
        return F.interpolate(x, scale_factor=self.scale_factor)


In [606]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.conv1=nn.Conv2d(2, 16, 3, stride=2, padding=1)
        self.conv2=nn.Conv2d(16, 32, 3, stride=2, padding=1)
        self.conv3=nn.Conv2d(32, 64, 3)
            
        self.deconv1=nn.ConvTranspose2d(64, 32, 3)
        self.deconv2=nn.ConvTranspose2d(32, 16, 3, stride=(2,2), padding=1, output_padding=1)
        self.deconv3=nn.ConvTranspose2d(16, 2, 3, stride=(2,2), padding=1, output_padding=1)
        

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.interpolate(x, scale_factor=2)
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.interpolate(x, scale_factor=2)
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        x = F.interpolate(x, scale_factor=2)
        
        x = F.relu(self.deconv1(x))
        x = F.relu(self.deconv2(x))
        x = torch.sigmoid(self.deconv3(x))
        return x

In [607]:
from viz import amv_analysis as aa
from viz import dataframe_calculators as dfc 
import datetime
import pickle 
import metpy
import numpy as np
import torch
dict_path = '../data/interim/dictionaries/dataframes.pkl'
dataframes_dict = pickle.load(open(dict_path, 'rb'))


start_date=datetime.datetime(2006,7,1,6,0,0,0)
end_date=datetime.datetime(2006,7,1,7,0,0,0)
df0 = aa.df_concatenator(dataframes_dict, start_date, end_date, False, True)

#df=df.dropna()
df=df0.loc[end_date:end_date]

lon = df.pivot('y', 'x', 'lon').values
lat = df.pivot('y', 'x', 'lat').values
u=df.pivot('y', 'x', 'u_scaled_approx').values
v=df.pivot('y', 'x', 'v_scaled_approx').values
u=np.nan_to_num(u)
v=np.nan_to_num(v)
qv=df.pivot('y', 'x', 'qv').values*100
qv=np.nan_to_num(qv)
qv=torch.from_numpy(qv)

ut=df.pivot('y', 'x', 'u').values
vt=df.pivot('y', 'x', 'v').values
ut=np.nan_to_num(ut)
vt=np.nan_to_num(vt)

speed_error=np.sqrt((u-ut)**2+(v-vt)**2)
print('speed_error')
print(speed_error.mean())
ut=torch.from_numpy(ut)
vt=torch.from_numpy(vt)


df=df0.loc[start_date:start_date]
u0=df.pivot('y', 'x', 'u_scaled_approx').values
v0=df.pivot('y', 'x', 'v_scaled_approx').values
u0=np.nan_to_num(u0)
v0=np.nan_to_num(v0)
qv0=df.pivot('y', 'x', 'qv').values*100
qv0=np.nan_to_num(qv0)
qv0=torch.from_numpy(qv0)

u0t=df.pivot('y', 'x', 'u').values
v0t=df.pivot('y', 'x', 'v').values
u0t=np.nan_to_num(u0t)
v0t=np.nan_to_num(v0t)
u0t=torch.from_numpy(u0t)
v0t=torch.from_numpy(v0t)

dx, dy = metpy.calc.lat_lon_grid_deltas(lon, lat)
dx=dx.magnitude 
mask = np.isnan(dx)
dx[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), dx[~mask])


dy=dy.magnitude 
mask = np.isnan(dy)
dy[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), dy[~mask])


mask = dx<1e-15
dx[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), dx[~mask])

mask = dy<1e-15
dy[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), dy[~mask])

dx=torch.from_numpy(dx)
dy=torch.from_numpy(dy)
dx_1=torch.rand(dx.shape)
dx_1[:,:]=1
dy_1=torch.rand(dy.shape)
dy_1[:,:]=1

hello world
concatenating dataframes for all dates for further analysis:
2006-07-01 06:00:00
2006-07-01 06:00:00
2006-07-01 07:00:00
2006-07-01 07:00:00
speed_error
2.9248843


In [608]:

#wind0=torch.rand([1,4,lon.shape[0],lon.shape[1]])
wind=torch.rand([1,4,lon.shape[0],lon.shape[1]])
wind[0,0,:,:]=torch.from_numpy(u0)
wind[0,1,:,:]=torch.from_numpy(v0)
wind[0,2,:,:]=torch.from_numpy(u)
wind[0,3,:,:]=torch.from_numpy(v)

winds=torch.rand([1,2,lon.shape[0],lon.shape[1]])
winds[0,0,:,:]=torch.from_numpy(u)
winds[0,1,:,:]=torch.from_numpy(v)
dt=3600
print(wind.shape)
dt=3600

torch.Size([1, 4, 360, 720])


In [609]:
def diff_central_x(dx, f):
    diff=torch.zeros((f.shape[0],f.shape[1]-2))
  
    for i,_ in enumerate(f): 
        x_diff = dx[i,1:] + dx[i,:-1]
        f_diff = f[i,2:] - f[i,:-2]
        diff[i,:]= f_diff/x_diff
    difft=torch.zeros(f.shape)
    difft[:,1:-1]=diff
    return difft


def diff_central_y(dy, f):
    diff=torch.zeros((f.shape[0]-2,f.shape[1]))
  
    for i,_ in enumerate(f.T): 
        y_diff = dy[1:,i] + dy[:-1,i]
        f_diff = f[2:,i] - f[:-2,i]
        diff[:,i]= f_diff/y_diff
    difft=torch.zeros(f.shape)
    difft[1:-1,:]=diff
    return difft

def omega(dx,dy,u,v):
    omega=diff_central_x(dx,v)-diff_central_y(dy,u)
    return omega

def advection(dx,dy,u,v,q):
    adv=diff_central_x(dx,(q*u))+diff_central_y(dy,(q*v))
    return adv


def loss_physics(wind,qv, model,dt):
    wind_cnn=wind+model.forward(wind)
    u=wind_cnn[:,0,:,:]
    v=wind_cnn[:,1,:,:]
    
    

    func=(u-ut)**2+(v-vt)**2
    return abs(func.mean())

def loss_physics0(wind0,wind,qv, model,dt):
    wind_cnn=wind+model.forward(wind)
    u=wind_cnn[:,2,:,:]
    v=wind_cnn[:,3,:,:]
    ug=wind[:,2,:,:]
    vg=wind[:,3,:,:]
    u=torch.squeeze(u)
    v=torch.squeeze(v)
    ug=torch.squeeze(ug)
    vg=torch.squeeze(vg)
    
    u0=wind_cnn[:,0,:,:]
    v0=wind_cnn[:,1,:,:]
    ug0=wind[:,0,:,:]
    vg0=wind[:,1,:,:]
    u0=torch.squeeze(u0)
    v0=torch.squeeze(v0)
    ug0=torch.squeeze(ug0)
    vg0=torch.squeeze(vg0)
    vor0 = omega(dx,dy,u0,v0)
    vor = omega(dx,dy,u,v)
    
    vor_adv = advection(dx,dy,u0,v0,vor0)
    
    dqdx=diff_central_x(dx_1, qv)
    dqdy=diff_central_y(dy_1, qv)
    dqdx0=diff_central_x(dx_1, qv0)
    dqdy0=diff_central_y(dy_1, qv0)

    reg=dqdx*dqdx*(u-ug)**2+dqdy*dqdy*(v-vg)**2
    reg0=dqdx0*dqdx0*(u0-ug0)**2+dqdy0*dqdy0*(v0-vg0)**2
    barot=vor-vor0-(vor_adv)*dt
    
    #func=10*(reg)+2e7*barot
    #func=5e3*reg
    func=1e4*(reg+reg0)+2e7*barot
    return abs(func.mean())

In [610]:
def train(model, num_epochs=5, batch_size=1, learning_rate=1e-2):
    torch.manual_seed(42)
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate, 
                                 weight_decay=1e-5) # <--

    outputs = []
    
    for epoch in range(num_epochs):
        loss = loss_physics(winds,qv, model,dt)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
        #outputs.append((epoch, img, recon),)
    #return outputs

In [611]:
model = Autoencoder()
#model=Autoenc()
max_epochs = 400
winds.size()

outputs = train(model, num_epochs=max_epochs)

Epoch:1, Loss:14.6178
Epoch:2, Loss:14.3984
Epoch:3, Loss:14.2383
Epoch:4, Loss:14.0295
Epoch:5, Loss:13.8255
Epoch:6, Loss:13.7293
Epoch:7, Loss:13.6311
Epoch:8, Loss:13.5765
Epoch:9, Loss:13.5353
Epoch:10, Loss:13.4399
Epoch:11, Loss:13.3657
Epoch:12, Loss:13.2961
Epoch:13, Loss:13.2595
Epoch:14, Loss:13.2076
Epoch:15, Loss:13.1678
Epoch:16, Loss:13.1247
Epoch:17, Loss:13.0741
Epoch:18, Loss:13.0397
Epoch:19, Loss:13.0067
Epoch:20, Loss:12.9884
Epoch:21, Loss:12.9677
Epoch:22, Loss:12.9567
Epoch:23, Loss:12.9278
Epoch:24, Loss:12.9316
Epoch:25, Loss:12.9163
Epoch:26, Loss:12.8818
Epoch:27, Loss:12.9075
Epoch:28, Loss:12.8581
Epoch:29, Loss:12.8382
Epoch:30, Loss:12.8013
Epoch:31, Loss:12.8297
Epoch:32, Loss:12.8066
Epoch:33, Loss:12.8014
Epoch:34, Loss:12.7436
Epoch:35, Loss:12.7504
Epoch:36, Loss:12.7117
Epoch:37, Loss:12.6919
Epoch:38, Loss:12.6505
Epoch:39, Loss:12.6472
Epoch:40, Loss:12.6848
Epoch:41, Loss:12.6664
Epoch:42, Loss:12.6237
Epoch:43, Loss:12.6727
Epoch:44, Loss:12.56

Epoch:347, Loss:11.7192
Epoch:348, Loss:11.6921
Epoch:349, Loss:11.7102
Epoch:350, Loss:11.6872
Epoch:351, Loss:11.6880
Epoch:352, Loss:11.6776
Epoch:353, Loss:11.6784
Epoch:354, Loss:11.6652
Epoch:355, Loss:11.6618
Epoch:356, Loss:11.6616
Epoch:357, Loss:11.6572
Epoch:358, Loss:11.6528
Epoch:359, Loss:11.6502
Epoch:360, Loss:11.6488
Epoch:361, Loss:11.6489
Epoch:362, Loss:11.6473
Epoch:363, Loss:11.6457
Epoch:364, Loss:11.6412
Epoch:365, Loss:11.6388
Epoch:366, Loss:11.6380
Epoch:367, Loss:11.6365
Epoch:368, Loss:11.6362
Epoch:369, Loss:11.6345
Epoch:370, Loss:11.6326
Epoch:371, Loss:11.6311
Epoch:372, Loss:11.6286
Epoch:373, Loss:11.6271
Epoch:374, Loss:11.6267
Epoch:375, Loss:11.6255
Epoch:376, Loss:11.6250
Epoch:377, Loss:11.6254
Epoch:378, Loss:11.6326
Epoch:379, Loss:11.6475
Epoch:380, Loss:11.6584
Epoch:381, Loss:11.6441
Epoch:382, Loss:11.6494
Epoch:383, Loss:11.6585
Epoch:384, Loss:11.6871
Epoch:385, Loss:11.6532
Epoch:386, Loss:11.6493
Epoch:387, Loss:11.6813
Epoch:388, Loss:

In [612]:
model.forward(winds)

tensor([[[[2.6899e-02, 4.6902e-03, 4.4163e-03,  ..., 3.3433e-02,
           1.2846e-01, 1.2763e-01],
          [1.8609e-03, 2.6646e-04, 8.1848e-04,  ..., 3.4463e-02,
           7.1362e-02, 1.0795e-01],
          [1.9133e-03, 5.3507e-04, 2.9154e-03,  ..., 3.5507e-02,
           2.9014e-02, 3.3214e-02],
          ...,
          [9.9010e-01, 9.9985e-01, 9.9988e-01,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          [8.7225e-01, 9.7525e-01, 9.7479e-01,  ..., 1.0000e+00,
           1.0000e+00, 9.9999e-01],
          [7.8089e-01, 9.5732e-01, 9.3480e-01,  ..., 1.0000e+00,
           9.9996e-01, 9.9988e-01]],

         [[1.4080e-01, 7.3020e-02, 5.7864e-02,  ..., 7.6140e-02,
           1.9053e-01, 2.0570e-01],
          [8.1759e-02, 7.1416e-02, 2.2829e-02,  ..., 5.6646e-03,
           6.3555e-03, 8.9281e-02],
          [1.1589e-01, 6.0958e-02, 3.8807e-02,  ..., 2.6360e-05,
           1.6533e-03, 1.3683e-03],
          ...,
          [4.1665e-02, 7.0464e-03, 5.3399e-02,  ..., 9.9915

In [613]:

wind_cnn=winds+model.forward(winds)
u=wind_cnn[:,0,:,:]
v=wind_cnn[:,1,:,:]
u=torch.squeeze(u)
v=torch.squeeze(v)



speed_error=torch.sqrt((u-ut)**2+(v-vt)**2)
print('speed_error')
print(speed_error.mean())

speed_error
tensor(2.5737, grad_fn=<MeanBackward0>)


In [614]:
dx=torch.rand([10,9])
dy=torch.rand([9,10])

dx[:,:]=2
dy[:,:]=2

f=torch.zeros([10,10])
f[:,:5]=1
print(f)
print(diff_central_x(dx,f))



tensor([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000, -0.2500, -0.2500,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.2500, -0.2500,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.2500, -0.2500,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.2500, -0.2500,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.2500, -0.2500, 

In [615]:
 torch.cuda.is_available()


False